In [ ]:
import numpy as np
import pandas as pd

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
path= '../input/tabular-playground-series-feb-2021/'
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
ss = pd.read_csv(path + 'sample_submission.csv')

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train.describe()

In [ ]:
test.describe()

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
cat_cols = ['cat{}'.format(i) for i in range(10)]
cont_cols = ['cont{}'.format(i) for i in range(14)]

train_data = train[cat_cols+cont_cols]
train_target = train['target']
test_data = test.drop('id',axis=1)

In [ ]:
train_data.shape, train_target.shape

In [ ]:
# continuous - standard scaling

st_scaler = StandardScaler()
st_scaler.fit(train_data[cont_cols])
train_data[cont_cols] = st_scaler.transform(train_data[cont_cols])
test_data[cont_cols] = st_scaler.transform(test_data[cont_cols])

In [ ]:
# categorical -> one-hot encoding

oh_enc = OneHotEncoder()
oh_enc.fit(train_data[cat_cols])
oh_cols = oh_enc.get_feature_names(cat_cols)

train_data[oh_cols] = oh_enc.transform(train_data[cat_cols]).toarray()
test_data[oh_cols] = oh_enc.transform(test_data[cat_cols]).toarray()

In [ ]:
train_data.drop(cat_cols, axis=1, inplace=True)
test_data.drop(cat_cols, axis=1, inplace=True)

In [ ]:
train_data

In [ ]:
test_data

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train_data, train_target, test_size=0.2)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

In [ ]:
lgb_reg = LGBMRegressor(n_estimators=1000, learning_rate = 0.05)
lgb_reg.fit(X_train, y_train, eval_set=[(X_val,y_val)], eval_metric='mse', early_stopping_rounds=200, verbose=50)
pred = lgb_reg.predict(X_val)

print(mean_squared_error(pred, y_val))

In [ ]:
ss.target = lgb_reg.predict(test_data)

In [ ]:
ss

In [ ]:
ss.to_csv('submission.csv', index=False)